In [1]:
import tensorflow as tf

file_url = 'https://github.com/PacktWorkshops/The-Deep-Learning-Workshop/blob/master/Chapter03/Datasets/Activity3.02/fruits360.zip'

zip_dir = tf.keras.utils.get_file('fruits360.zip', origin=file_url, extract=True)


In [2]:
import pathlib

path = pathlib.Path(zip_dir).parent / 'fruits360_filtered'
train_dir = path / 'Training'
test_dir = path / 'Test'

total_train = 11398
total_val = 4752

In [3]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_data_gen = ImageDataGenerator(rescale=1./255, rotation_range=40, width_shift_range=0.1, height_shift_range=0.1, \
                                   shear_range=0.2, zoom_range=0.2, horizontal_flip=True, fill_mode='nearest')

val_data_gen =ImageDataGenerator(rescale=1./255)


batch_size = 16
img_height = 100
img_width = 100
channels = 3

train_image_gen = train_data_gen.flow_from_directory(directory=train_dir, batch_size=batch_size, target_size=(img_height, img_width))
val_image_gen = val_data_gen.flow_from_directory(directory=test_dir, batch_size=batch_size, target_size=(img_height, img_width))



Found 11398 images belonging to 120 classes.
Found 4752 images belonging to 120 classes.


In [4]:
from tensorflow.keras.applications import VGG16
import os
os.environ['KMP_DUPLICATE_LIB_OK']='True'
base_model = VGG16(input_shape=(img_height, img_width, channels), weights='imagenet', include_top=False)

base_model.trainable = False

sec_last_layer = tf.keras.layers.Dense(1000, activation='relu')
prediction_layer = tf.keras.layers.Dense(120, activation='softmax')

new_model = tf.keras.Sequential([base_model, tf.keras.layers.Flatten(), sec_last_layer, prediction_layer])

optimizer = tf.keras.optimizers.Adam(0.001)

new_model.compile(loss ='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])
new_model.summary()
base_model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Model)                (None, 3, 3, 512)         14714688  
_________________________________________________________________
flatten (Flatten)            (None, 4608)              0         
_________________________________________________________________
dense (Dense)                (None, 1000)              4609000   
_________________________________________________________________
dense_1 (Dense)              (None, 120)               120120    
Total params: 19,443,808
Trainable params: 4,729,120
Non-trainable params: 14,714,688
_________________________________________________________________
Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 100, 100, 3)]     0         
_____________________

In [5]:
new_model.fit_generator(train_image_gen, epochs=5, steps_per_epoch=11398//batch_size, validation_data=val_image_gen, \
                    validation_steps=4752//batch_size)

Epoch 1/5
  9/712 [..............................] - ETA: 1:09:24 - loss: 5.5791 - accuracy: 0.0347

KeyboardInterrupt: 